In [1]:
"""
Capstone 3 - Clinical Insights Assistant
Notebook: 05_genai_summary_generation.ipynb
------------------------------------------
Generative AI summary examples:
- Summarize doctor notes (GenAI)
- Generate regulatory (FDA-style) summaries
- Preview & save outputs
"""

import os
import sys
import json
import pandas as pd
from pathlib import Path
import datetime

# plotting / display niceties
from IPython.display import display, Markdown

# make sure src is importable
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../src")))

# import genai wrapper (gemini/openai wrapper implemented earlier)
try:
    from genai_interface import summarize_doctor_notes, generate_regulatory_summary
except Exception as e:
    raise ImportError(
        "Could not import genai_interface. Ensure src/genai_interface.py exists and is valid. Error: "
        + str(e)
    )

# ------------------- CONFIG -------------------
pd.set_option("display.max_colwidth", 200)

# locate data file robustly
project_root = os.getcwd()
if "notebooks" in project_root:
    DATA_PATH = os.path.abspath(os.path.join(project_root, "../data/clinical_trial_data.csv"))
else:
    DATA_PATH = os.path.abspath(os.path.join(project_root, "data/clinical_trial_data.csv"))

print("Using dataset:", DATA_PATH)

# load dataset
df = pd.read_csv(DATA_PATH, parse_dates=["visit_date"])
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# ------------------- Utility functions -------------------
def preview_notes(df, n=20):
    notes = df["doctor_notes"].dropna().unique().tolist()
    return notes[:n]

def save_output(text, name_prefix="genai_output"):
    out_dir = Path(os.path.abspath(os.path.join(os.getcwd(), "../outputs")))
    out_dir.mkdir(parents=True, exist_ok=True)
    ts = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
    filename = out_dir / f"{name_prefix}_{ts}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)
    return str(filename)

# ------------------- 1) Preview doctor notes -------------------
display(Markdown("## 1) Doctor Notes Preview"))
num_preview = 30
notes_preview = preview_notes(df, n=num_preview)
print(f"Showing first {len(notes_preview)} unique notes (out of {df['doctor_notes'].dropna().nunique()} unique notes):\n")
for i, ntext in enumerate(notes_preview, 1):
    print(f"{i}. {ntext}")

# ------------------- 2) Summarize doctor notes (GenAI) -------------------
display(Markdown("## 2) Generate Doctor Notes Summary (GenAI)"))
print("Configure how many notes to include for summarization and then call the function.")

# user-configurable: how many notes to include
N_NOTES = 100   # change as needed in notebook before running
notes_for_summary = df["doctor_notes"].dropna().unique().tolist()[:N_NOTES]

print(f"Including {len(notes_for_summary)} notes for summarization (N_NOTES={N_NOTES}).")
display(Markdown("### Sample notes used (first 10):"))
for s in notes_for_summary[:10]:
    print("-", s)

# call summarization safely and display result
try:
    summary_text = summarize_doctor_notes(notes_for_summary)
    display(Markdown("### ✨ Doctor Notes Summary (GenAI Output)"))
    print(summary_text)
    # Save
    saved_path = save_output(summary_text, "doctor_notes_summary")
    print(f"\nSaved summary to: {saved_path}")
except Exception as e:
    display(Markdown("### ❗ Summarization Error"))
    print("Error calling the GenAI summarizer:", str(e))
    print("Check that your GOOGLE_API_KEY (or OpenAI key) is set in .env and genai_interface is configured.")

# ------------------- 3) Generate regulatory summary from trial text -------------------
display(Markdown("## 3) Generate Regulatory (FDA-style) Summary"))

# Compose a concise trial-results text automatically (or paste your own below)
auto_trial_text = []
auto_trial_text.append(f"Dataset rows: {len(df)}")
if "cohort" in df.columns:
    cohort_counts = df["cohort"].value_counts().to_dict()
    auto_trial_text.append(f"Cohort counts: {cohort_counts}")
# Compliance overview
if "compliance_pct" in df.columns or "compliance_rate" in df.columns:
    col = "compliance_pct" if "compliance_pct" in df.columns else "compliance_rate"
    auto_trial_text.append(f"Mean compliance: {df[col].mean():.2f}")
# Adverse events
if "adverse_event_flag" in df.columns or "adverse_event" in df.columns:
    a_col = "adverse_event_flag" if "adverse_event_flag" in df.columns else "adverse_event"
    auto_trial_text.append(f"Adverse event count: {int(df[a_col].sum())}")
# Outcome summary
if "outcome_score" in df.columns:
    auto_trial_text.append(f"Mean outcome score: {df['outcome_score'].mean():.2f}")
trial_text = "\n".join(auto_trial_text)

display(Markdown("### Auto-generated trial summary (you can edit this before generation):"))
print(trial_text)

# Option: override trial_text manually by editing the variable below (in Notebook)
# trial_text = "Paste or write your trial results summary here..."

try:
    regulatory_summary = generate_regulatory_summary(trial_text)
    display(Markdown("### 🧾 Regulatory Summary (GenAI Output)"))
    print(regulatory_summary)
    saved_path2 = save_output(regulatory_summary, "regulatory_summary")
    print(f"\nSaved regulatory summary to: {saved_path2}")
except Exception as e:
    display(Markdown("### ❗ Regulatory Summary Error"))
    print("Error calling the GenAI regulatory summary generator:", str(e))
    print("Make sure your API key is set in .env and genai_interface is using it correctly.")

# ------------------- 4) Quick examples & tips -------------------
display(Markdown("## 4) Tips & Next Steps"))
print("- If API calls fail, ensure .env contains GOOGLE_API_KEY and genai_interface uses python-dotenv.")
print("- For larger sets of notes, you may want to batch notes into chunks and summarize each chunk, then aggregate.")
print("- Consider adding prompt templates in src/genai_interface.py for more controlled/regulatory-safe outputs.")
print("- Save generated summaries into your `outputs/` folder for versioning and review.")

# ------------------- 5) Show saved outputs directory -------------------
out_dir = Path(os.path.abspath(os.path.join(os.getcwd(), "../outputs")))
if out_dir.exists():
    print("\nSaved output files:")
    for p in sorted(out_dir.glob("*.txt")):
        print("-", p.name)
else:
    print("\nNo saved outputs yet. Summaries will be saved to ../outputs when generated.")


Using dataset: c:\Users\mackrish_malik\Desktop\clinical-insights-assistant\data\clinical_trial_data.csv
Dataset loaded: 6000 rows, 9 columns


## 1) Doctor Notes Preview

Showing first 5 unique notes (out of 5 unique notes):

1. Patient stable, no complaints.
2. Adverse reaction observed, dosage adjustment needed.
3. Mild headache reported, advised rest.
4. Fatigue noted, monitoring ongoing.
5. Symptoms improving with current dosage.


## 2) Generate Doctor Notes Summary (GenAI)

Configure how many notes to include for summarization and then call the function.
Including 5 notes for summarization (N_NOTES=100).


...

Dataset rows: 6000
Cohort counts: {'A': 3030, 'B': 2970}
Mean compliance: 89.27
Adverse event count: 623
Mean outcome score: 83.24


### 🧾 Regulatory Summary (GenAI Output)

This randomized, controlled trial evaluated the efficacy and safety of a novel intervention in 6000 participants. The study population was split into two cohorts, A (n=3030) and B (n=2970). High treatment compliance was observed across both cohorts, with an overall mean compliance rate of 89.27%. This high level of adherence correlated with a statistically significant improvement in patient outcomes, as measured by a mean outcome score of 83.24 across the entire study population. These results suggest that the intervention, when consistently adhered to, demonstrates a beneficial effect on the targeted clinical outcome.

While the intervention demonstrated a positive impact on patient outcomes, the study also recorded a total of 623 adverse events across all participants. Further analysis is required to determine the specific nature, severity, and causality of these adverse events in relation to the intervention. Detailed characterization of these events, including frequency and potenti

## 4) Tips & Next Steps

- If API calls fail, ensure .env contains GOOGLE_API_KEY and genai_interface uses python-dotenv.
- For larger sets of notes, you may want to batch notes into chunks and summarize each chunk, then aggregate.
- Consider adding prompt templates in src/genai_interface.py for more controlled/regulatory-safe outputs.
- Save generated summaries into your `outputs/` folder for versioning and review.

Saved output files:
- doctor_notes_summary_20251017T072814Z.txt
- regulatory_summary_20251017T072817Z.txt
